## Libraries

In [1]:
import numpy as np
import pandas as pd
#from os import path
import re
import string

#NLTK
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.util import ngrams
from nltk import bigrams, trigrams
import nltk
from nltk.util import ngrams

#sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import validation_curve
#from sklearn.metrics import zero_one_score

#visualization
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
#% matplotlib inline


#from sklearn.cross_validation import train_test_split
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
#from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, ArrayDictionary, StopWordRemover


#https://towardsdatascience.com/multiclass-classification-with-support-vector-machines-svm-kernel-trick-kernel-functions-f9d5377d6f02
#https://www.analyseup.com/python-machine-learning/stratified-kfold.html
#https://www.pluralsight.com/guides/validating-machine-learning-models-scikit-learn
#https://www.pengalaman-edukasi.com/2020/04/apa-itu-k-fold-cross-validation.html
#https://chrisalbon.com/code/machine_learning/support_vector_machines/svc_parameters_using_rbf_kernel/
#https://www.analyseup.com/python-machine-learning/stratified-kfold.html
#https://stats.stackexchange.com/questions/462363/creating-a-dataframe-includes-my-cross-validation-scores

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("C:/Users/USER/anaconda3/Lib/site-packages")

## Data

In [4]:
df = pd.read_csv("data_tweet_finall_preprocess_fix2.csv")
df = df.drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)
df.head()

,Topic,Tweet,Class,Author,Normalized,Stopword,Stemmed,Tokenize
0,agama,rakyat menunggu keseriusan pernyataan soal pen...,y,Bhrigutantra,rakyat menunggu keseriusan pernyataan soal pen...,rakyat menunggu keseriusan pernyataan penegaka...,rakyat tunggu serius nyata tega hukum detik la...,"['rakyat', 'tunggu', 'serius', 'nyata', 'tega'..."
1,agama,baca dong klo gatau apa gausah sok bawa agama...,y,kckiranaaa,baca dong kalau tak tau apa tidak usah sok ba...,baca sok bawa agama komen sesuai pendapatmu,baca sok bawa agama komen sesuai dapat,"['baca', 'sok', 'bawa', 'agama', 'komen', 'ses..."
2,agama,skg tokoh agama lebih memilih jabatan dan gol...,y,Lani34823852,sekarang tokoh agama lebih memilih jabatan da...,tokoh agama memilih jabatan golongan,tokoh agama pilih jabat golong,"['tokoh', 'agama', 'pilih', 'jabat', 'golong']"
3,agama,inilah pentingnya membaca biar gasotoy menjus...,y,KCangcut,inilah pentingnya membaca biar gasotoy menjus...,membaca gasotoy menjustifikasi agama,baca gasotoy justifikasi agama,"['baca', 'gasotoy', 'justifikasi', 'agama']"
4,agama,hot take orang yg dikit bawa zodiak ke topik a...,y,Far_away90,hot take orang yang sedikit bawa zodiak ke top...,hot take orang bawa zodiak topik apapun menyeb...,hot take orang bawa zodiak topik apa sebal ora...,"['hot', 'take', 'orang', 'bawa', 'zodiak', 'to..."


In [5]:
df_class = pd.DataFrame(df['Class'])
df_class.head()

,Class
0,y
1,y
2,y
3,y
4,y


# Fitur = 10k

In [17]:
vect = CountVectorizer()
X = vect.fit_transform(df['Stemmed'].values.astype('U'))
bow_unigram = pd.DataFrame(X.todense(), columns = vect.get_feature_names())
bow_unigram.head()

,aan,aanya,abad,abadi,abai,abal,abang,abangda,abas,abbas,...,zulkarnain,zulpan,zura,zvt,zxtle,zyafhoja,zyan,zyandaru,zyk,zyn
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#Label encoder

labelencoder = LabelEncoder()
df_class['Class'] = labelencoder.fit_transform(df_class['Class'])
df_class.head()

,Class
0,1
1,1
2,1
3,1
4,1


### Baseline SVM

In [12]:
#SVM

def SVM(x,y,z):
    
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=z)
    svc = LinearSVC(C=0.1)
    svc.fit(X_train, Y_train)
    y_pred = svc.predict(X_test)
    
    
    class_report = classification_report(Y_test, y_pred)
    matrik = confusion_matrix(Y_test, y_pred)
    akurasi = accuracy_score(Y_test, y_pred)
    presisi = precision_score(Y_test, y_pred)
    recall = recall_score(Y_test, y_pred)
    f1score = f1_score(Y_test, y_pred)
    print("Confusion Matrix : \n",matrik)
    print("Classification Report : \n", class_report)
    print("Accuracy  : ", akurasi)
    print("Precision : ",presisi)
    print("Recall    : ",recall)
    print("F1 Score  : ",f1score)
    return None

In [13]:
rasio = 0.1
print('=========================== SVM ===========================')
print(" ")
print("Hasil ke- 1")
SVM(bow_unigram.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 2")
SVM(bow_unigram.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 3")
SVM(bow_unigram.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 4")
SVM(bow_unigram.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 5")
SVM(bow_unigram.to_numpy(),df_class,rasio)

=========================== SVM ===========================
 
Hasil ke- 1
Confusion Matrix : 
 [[922 119]
 [138 879]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.87      0.89      0.88      1041
           1       0.88      0.86      0.87      1017

    accuracy                           0.88      2058
   macro avg       0.88      0.87      0.88      2058
weighted avg       0.88      0.88      0.88      2058

Accuracy  :  0.8751214771622935
Precision :  0.8807615230460922
Recall    :  0.8643067846607669
F1 Score  :  0.8724565756823822

Hasil ke- 2
Confusion Matrix : 
 [[918 121]
 [157 862]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.85      0.88      0.87      1039
           1       0.88      0.85      0.86      1019

    accuracy                           0.86      2058
   macro avg       0.87      0.86      0.86      2058
weighted avg       0.87      0.86      0.86   

In [33]:

def RF(x,y,z):
    
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=z)
    RandomForest = RandomForestClassifier(min_samples_leaf=3)
    RandomForest.fit(X_train, Y_train)
    y_pred = RandomForest.predict(X_test)
    y_true = Y_test
    
    class_report = classification_report(y_true, y_pred)
    matrik = confusion_matrix(y_true, y_pred)
    akurasi = accuracy_score(y_true, y_pred)
    presisi = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1score = f1_score(y_true, y_pred)
    print("Confusion Matrix : \n",matrik)
    print("Classification Report : \n", class_report)
    print("Accuracy  : ", akurasi)
    print("Precision : ",presisi)
    print("Recall    : ",recall)
    print("F1 Score  : ",f1score)
    return None

In [34]:
rasio = 0.1
print('=========================== Random Forest ===========================')
print(" ")
print("Hasil ke- 1")
RF(bow_unigram.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 2")
RF(bow_unigram.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 3")
RF(bow_unigram.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 4")
RF(bow_unigram.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 5")
RF(bow_unigram.to_numpy(),df_class,rasio)

=========================== Random Forest ===========================
 
Hasil ke- 1
Confusion Matrix : 
 [[909 124]
 [143 882]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.86      0.88      0.87      1033
           1       0.88      0.86      0.87      1025

    accuracy                           0.87      2058
   macro avg       0.87      0.87      0.87      2058
weighted avg       0.87      0.87      0.87      2058

Accuracy  :  0.8702623906705539
Precision :  0.8767395626242545
Recall    :  0.8604878048780488
F1 Score  :  0.8685376661742984

Hasil ke- 2
Confusion Matrix : 
 [[979 114]
 [167 798]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.85      0.90      0.87      1093
           1       0.88      0.83      0.85       965

    accuracy                           0.86      2058
   macro avg       0.86      0.86      0.86      2058
weighted avg       0.86      0.86   

## Semua Fitur 

In [6]:
vect = CountVectorizer()
X = vect.fit_transform(df['Stemmed'].values.astype('U'))
bow_unigram_all = pd.DataFrame(X.todense(), columns = vect.get_feature_names())
bow_unigram_all.head()

,aan,aanya,abad,abadi,abai,abal,abang,abangda,abas,abbas,...,zulkarnain,zulpan,zura,zvt,zxtle,zyafhoja,zyan,zyandaru,zyk,zyn
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#Label encoder

labelencoder = LabelEncoder()
df_class['Class'] = labelencoder.fit_transform(df_class['Class'])
df_class.head()

,Class
0,1
1,1
2,1
3,1
4,1


### Baseline SVM

In [34]:
#SVM

def SVM(x,y,z):
    
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=z)
    svc = LinearSVC(C=0.1)
    svc.fit(X_train, Y_train)
    y_pred = svc.predict(X_test)
    y_true = Y_test
    
    class_report = classification_report(y_true, y_pred)
    matrik = confusion_matrix(y_true, y_pred)
    akurasi = accuracy_score(y_true, y_pred)
    presisi = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1score = f1_score(y_true, y_pred)
    print("Confusion Matrix : \n",matrik)
    print("Classification Report : \n", class_report)
    print("Accuracy  : ", akurasi)
    print("Precision : ",presisi)
    print("Recall    : ",recall)
    print("F1 Score  : ",f1score)
    return None

In [9]:
rasio = 0.1
print('=========================== SVM ===========================')
print(" ")
print("Hasil ke- 1")
SVM(bow_unigram_all.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 2")
SVM(bow_unigram_all.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 3")
SVM(bow_unigram_all.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 4")
SVM(bow_unigram_all.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 5")
SVM(bow_unigram_all.to_numpy(),df_class,rasio)

=========================== SVM ===========================
 
Hasil ke- 1
Confusion Matrix : 
 [[923 141]
 [158 836]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.85      0.87      0.86      1064
           1       0.86      0.84      0.85       994

    accuracy                           0.85      2058
   macro avg       0.85      0.85      0.85      2058
weighted avg       0.85      0.85      0.85      2058

Accuracy  :  0.8547133138969873
Precision :  0.8556806550665302
Recall    :  0.8410462776659959
F1 Score  :  0.8483003551496702

Hasil ke- 2
Confusion Matrix : 
 [[948 122]
 [144 844]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.87      0.89      0.88      1070
           1       0.87      0.85      0.86       988

    accuracy                           0.87      2058
   macro avg       0.87      0.87      0.87      2058
weighted avg       0.87      0.87      0.87   

### Baseline Random Forest

In [36]:
#Random Forest

def RF(x,y,z):
    
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=z)
    RandomForest = RandomForestClassifier(random_state=0)
    RandomForest.fit(X_train, Y_train)
    y_pred = RandomForest.predict(X_test)
    y_true = Y_test
    
    class_report = classification_report(y_true, y_pred)
    matrik = confusion_matrix(y_true, y_pred)
    akurasi = accuracy_score(y_true, y_pred)
    presisi = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1score = f1_score(y_true, y_pred)
    print("Confusion Matrix : \n",matrik)
    print("Classification Report : \n", class_report)
    print("Accuracy  : ", akurasi)
    print("Precision : ",presisi)
    print("Recall    : ",recall)
    print("F1 Score  : ",f1score)
    return None

In [13]:
rasio = 0.1
print('=========================== Random Forest ===========================')
print(" ")
print("Hasil ke- 1")
RF(bow_unigram_all.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 2")
RF(bow_unigram_all.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 3")
RF(bow_unigram_all.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 4")
RF(bow_unigram_all.to_numpy(),df_class,rasio)
print("")
print("Hasil ke- 5")
RF(bow_unigram_all.to_numpy(),df_class,rasio)

=========================== Random Forest ===========================
 
Hasil ke- 1
Confusion Matrix : 
 [[937 130]
 [116 875]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.89      0.88      0.88      1067
           1       0.87      0.88      0.88       991

    accuracy                           0.88      2058
   macro avg       0.88      0.88      0.88      2058
weighted avg       0.88      0.88      0.88      2058

Accuracy  :  0.880466472303207
Precision :  0.8706467661691543
Recall    :  0.8829465186680121
F1 Score  :  0.8767535070140282

Hasil ke- 2
Confusion Matrix : 
 [[915 146]
 [124 873]]
Classification Report : 
               precision    recall  f1-score   support

           0       0.88      0.86      0.87      1061
           1       0.86      0.88      0.87       997

    accuracy                           0.87      2058
   macro avg       0.87      0.87      0.87      2058
weighted avg       0.87      0.87    